In [12]:
!pip install pandas torch transformers==4.33.0 tqdm sypy

ModuleNotFoundError: No module named 'pexpect'

In [13]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

In [14]:
import sympydd
print(sympy.core.numbers)

ModuleNotFoundError: No module named 'sympydd'

In [15]:
MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"  # Change if you use something else
INPUT_FILE = "kaggle_dataset.csv"   # Your Kaggle file
OUTPUT_FILE = "prompts_and_code.json"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
def load_model():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16
    )
    return tokenizer, model

def generate_prompt(tokenizer, model, code_snippet):
    prompt = f"""You are a helpful coding assistant.

Given the following code:

{code_snippet}

Please generate a realistic natural-language prompt that would cause an AI model to generate this code."""
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    result = decoded.split(prompt)[-1].strip()
    return result

def main():
    tokenizer, model = load_model()
    df = pd.read_csv(INPUT_FILE)
    
    print("GENERATING PROMPT")
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        code = row['code']
        label = row['label']  # Assuming 0 = clean, 1 = vulnerable
        
        try:
            generated_prompt = generate_prompt(tokenizer, model, code)
            results.append({
                "id": row['id'] if 'id' in row else idx,
                "prompt": generated_prompt,
                "code": code,
                "label": label
            })
        except Exception as e:
            print(f"Error on row {idx}: {e}")
            continue
        
        break

    print("GENERATED PROMPT")

    # Save
    pd.DataFrame(results).to_json(OUTPUT_FILE, orient="records", indent=2)

if __name__ == "__main__":
    main()

RuntimeError: Failed to import transformers.models.llama.modeling_llama because of the following error (look up to see its traceback):
No module named 'transformers.models.llama.modeling_llama'